In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('DataExtract_v4.csv') #read in CSV file provided

## Calc deal length in business days

In [3]:
#Convert to datetime
df['AssignedDate']= pd.to_datetime(df['AssignedDate'])
df['FirstDeliverableProvidedDate']= pd.to_datetime(df['FirstDeliverableProvidedDate'])

#Count business days between dates
A = [d.date() for d in df['AssignedDate']]
B = [d.date() for d in df['FirstDeliverableProvidedDate']]

df['days_for_deal'] = np.busday_count(A, B, weekmask='1111100')

## Set negative deal length to 0 and create cap for deal length at 25 days

In [4]:
#replace negative days for deals and outliers
mask = df['days_for_deal'] < 0
df.loc[mask, 'days_for_deal'] = 0

#replace large outliers
mask = df['days_for_deal'] > 25
df.loc[mask, 'days_for_deal'] = 25

## Calc PSA tenure

In [5]:
unique_ids = df['OwnerID'].unique()

new_df = pd.DataFrame(columns= df.columns)

for num in unique_ids:
    temp = df[df['OwnerID']== num]
    temp = temp.sort_values(by=['AssignedDate'], ascending=False)
    for i in range(len(temp)):
        temp['Time in Job Profile'].iloc[i] = max(0,temp['Time in Job Profile'].iloc[0] - ((temp['AssignedDate'].iloc[0] - temp['AssignedDate'].iloc[i]).days/30/12))
    new_df = new_df.append(temp, ignore_index=True)

C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\User

C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\User

C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\User

C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\User

In [6]:
df = new_df.copy(deep=True)

## Number of live jobs per PSA

In [7]:
df['Jobs In Progress']=0

In [8]:
unique_ids = df['OwnerID'].unique()

new_df = pd.DataFrame(columns= df.columns)

for num in unique_ids:
    temp = df[df['OwnerID'] == num]
    temp = temp.sort_values(by=['AssignedDate'], ascending=True)
    for i in range(len(temp)):
        #temp['Job In Progress'].iloc[i]=0
        for j in range(i):
            if(temp['FirstDeliverableProvidedDate'].iloc[j]>temp['AssignedDate'].iloc[i]):
                temp['Jobs In Progress'].iloc[i]+=1
    new_df = new_df.append(temp, ignore_index=True)

C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\User

C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\User

C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\User

C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\geogh\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\User

In [9]:
df = new_df.copy(deep=True)

## Number of unique technologies per PSA per quarter

In [10]:
df['technology_count'] = df.groupby(["OwnerID", "Fiscal Year", "Fiscal Quarter"])['Technologyname'].transform('nunique')

## Number of Deals each PSA completes per month and quarter

In [11]:
#setting up columns for masking, these will be dropped
df['Year_FirstDeliverable'] = df['FirstDeliverableProvidedDate'].dt.year
df['MOY_FirstDeliverable'] = df['FirstDeliverableProvidedDate'].dt.month
df['Month+Year_FirstDeliverable'] = df['Year_FirstDeliverable'].astype(str) + " " + df['MOY_FirstDeliverable'].astype(str)
df['deals_completed_this_month']=0
df['deals_completed_this_quarter']=0

In [12]:
#calc deals each PSA completes per month
unique_ids = df['OwnerID'].unique()
unique_months = df['Month+Year_FirstDeliverable'].unique()

for i in unique_ids:
    for j in unique_months:
        mask = (df['TechnologySolutionOwnerBadge'] == i) & (df['Month+Year_FirstDeliverable'] == j)
        deals_completed = len(df[mask])
        df.loc[mask, 'deals_completed_this_month'] = deals_completed

In [13]:
#calc deals each PSA completes per quarter
unique_ids = df['OwnerID'].unique()
unique_quarters = df['Fiscal Quarter'].unique()

for i in unique_ids:
    for j in unique_quarters:
        mask = (df['TechnologySolutionOwnerBadge'] == i) & (df['Fiscal Quarter'] == j)
        deals_completed = len(df[mask])
        df.loc[mask, 'deals_completed_this_quarter'] = deals_completed

In [14]:
df.drop(columns=['Year_FirstDeliverable', 'MOY_FirstDeliverable', 'Month+Year_FirstDeliverable'], inplace=True)

## Flags for Deal Complexity: High, Medium, Low

In [15]:
df['deal_complexity'] = np.where(df['days_for_deal'] <= 1, '1. Low',
                                 np.where(df['days_for_deal'] <= 7, '2. Medium', '3. High'
                                        ))

## Output

In [16]:
#new columns
df.head().columns[-8:]

Index(['dealtechnologytags', 'Time in Job Profile', 'days_for_deal',
       'Jobs In Progress', 'technology_count', 'deals_completed_this_month',
       'deals_completed_this_quarter', 'deal_complexity'],
      dtype='object')

In [17]:
df.to_csv('DataExtract_v5.csv')